In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("insurance.csv")
df.rename(columns={"charges":"expenses"},inplace = True)
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
continuos_var = ['age','bmi','expenses']
discrete_var = ['sex','smoker','region']
count_var = ['children']

In [5]:
df[continuos_var].describe()

,age,bmi,expenses
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,13270.422265
std,14.049960,6.098187,12110.011237
min,18.000000,15.960000,1121.873900
25%,27.000000,26.296250,4740.287150
50%,39.000000,30.400000,9382.033000
75%,51.000000,34.693750,16639.912515
max,64.000000,53.130000,63770.428010


In [6]:
df[discrete_var].describe()

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364


In [7]:
df[continuos_var].corr()

,age,bmi,expenses
age,1.000000,0.109272,0.299008
bmi,0.109272,1.000000,0.198341
expenses,0.299008,0.198341,1.000000


In [8]:
df.drop_duplicates(inplace=True)
df.drop('region',axis = 1,inplace = True) # vaadu ekkada unte enti?

In [9]:
df['sex'].replace({'female':0,'male':1},inplace = True)
df['smoker'].replace({'no':0,'yes':1},inplace = True)

In [10]:
x = df.drop('expenses',axis = 1)
y = df['expenses']
df

,age,sex,bmi,children,smoker,expenses
0,19,0,27.900,0,1,16884.92400
1,18,1,33.770,1,0,1725.55230
2,28,1,33.000,3,0,4449.46200
3,33,1,22.705,0,0,21984.47061
4,32,1,28.880,0,0,3866.85520
...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830
1334,18,0,31.920,0,0,2205.98080
1335,18,0,36.850,0,0,1629.83350
1336,21,0,25.800,0,0,2007.94500


In [11]:
train_r2 = []
cv = []
test_r2 = []

for i in range(0,100):
    
    from sklearn.model_selection import train_test_split
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=i)
    
    from sklearn.linear_model import ElasticNet
    model=ElasticNet()
    model.fit(x_train,y_train)

    ypred_train = model.predict(x_train)
    ypred_test = model.predict(x_test)
    
    from sklearn.model_selection import cross_val_score
    cv.append(cross_val_score(model,x_train,y_train,cv=5).mean())
    
    from sklearn.metrics import r2_score
    train_r2.append(r2_score(y_train,ypred_train))
    test_r2.append(r2_score(y_test,ypred_test))
    
em =pd.DataFrame({"train_r2":train_r2,"cv":cv})

# for best random state number...just check train == cv or not

gm = em[(abs(em["train_r2"]-em["cv"])<=0.05)] 

# Difference b/w train and cv must be less than 0.05 itself

rs=gm[gm["cv"]==gm["cv"].max()].index.tolist()[0]

rs  #[it should be 9]

9

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)


In [13]:
from sklearn.model_selection import GridSearchCV
estimator = ElasticNet()

param_grid = {'alpha':list(range(1,101)),
             'l1_ratio':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}

modelhp = GridSearchCV(estimator,param_grid,cv=5,scoring='r2')
modelhp.fit(x_train,y_train)
modelhp.best_params_

{'alpha': 60, 'l1_ratio': 1}

# **At LASSO we got Alpha = 60, L1 means lambda,L1 = 1, Whichs means lasso is best**
- it is also indirectly saying that lasso is best
- L1 = 1 means 100% LASSO is suited, L1=0 means 100% RIDGE is suited
- if we get in b/w values...we hv to try algoriths lasso,ridge,lr...etc


In [14]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha = 60,l1_ratio = 1)
model.fit(x_train,y_train)

print("intercep values : ",model.intercept_)
print("coefficient values : ",model.coef_)

ypred_train = model.predict (x_train)
ypred_test = model.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

intercep values :  -12040.593571887146
coefficient values :  [  264.38161528    -0.           316.89561192   373.0709729
 23622.96074097]
cv_score : 0.7537058055900724
train_r2 : 0.7591801991925938
test_r2 : 0.7009201105342885


# **Second column slope is zero means remove that column(sex)**

In [15]:
x = x.drop('sex',axis = 1)
#df.drop('sex',axis = 1,inplace = True)
y = df['expenses']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)

from sklearn.linear_model import ElasticNet
model1 = ElasticNet(alpha = 60,l1_ratio=1)
model1.fit(x_train,y_train)

print("intercep values : ",model1.intercept_)
print("coefficient values : ",model1.coef_)

ypred_train = model1.predict (x_train)
ypred_test = model1.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model1,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

intercep values :  -12040.588905108545
coefficient values :  [  264.38141961   316.89570704   373.07104773 23622.96061177]
cv_score : 0.753814164737532
train_r2 : 0.7591801990707372
test_r2 : 0.7009201316340361


In [16]:
input_data = {'age':35,
             'sex':'male',
             'bmi':31.4,
             'children':5,
             'smoker':'yes',
             'region':'southeast'}
df_test =pd.DataFrame(input_data,index=[0])
df_test

,age,sex,bmi,children,smoker,region
0,35,male,31.4,5,yes,southeast


In [17]:
df_test.drop('region',axis = 1,inplace = True)
df_test['sex'].replace({'female':0,'male':1},inplace = True)
df_test['smoker'].replace({'no':0,'yes':1},inplace = True)
df_test.drop('sex',axis = 1,inplace = True)
df_test

,age,bmi,children,smoker
0,35,31.4,5,1


In [18]:
model1.predict(df_test)

array([32651.60183275])

In [19]:
import warnings
warnings.simplefilter("ignore")

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

# Assuming x and y are defined (features and target)
# x = dataframe[feature_columns]
# y = dataframe[target_column]

models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "K Neighbors": KNeighborsRegressor(),
    "XGBoost": XGBRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

# Metrics dictionaries
train_r2, test_r2, cv_score_train, train_rmse, test_rmse, train_mae, test_mae, train_mse, test_mse = {}, {}, {}, {}, {}, {}, {}, {}, {}
train_r2_pca, test_r2_pca, cv_score_train_pca, train_rmse_pca, test_rmse_pca, train_mae_pca, test_mae_pca, train_mse_pca, test_mse_pca = {}, {}, {}, {}, {}, {}, {}, {}, {}
best_params, best_random_state = {}, {}

for name, model in models.items():
    train_r2_temp, test_r2_temp, cv_temp = [], [], []
    
    for i in range(0, 100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=i)
        scaler = MinMaxScaler()
        x_train_scaled, x_test_scaled = scaler.fit_transform(x_train), scaler.transform(x_test)

        model.fit(x_train_scaled, y_train)
        ypred_train, ypred_test = model.predict(x_train_scaled), model.predict(x_test_scaled)

        cv_temp.append(cross_val_score(model, x_train_scaled, y_train, cv=5, scoring='r2').mean())
        train_r2_temp.append(r2_score(y_train, ypred_train))
        test_r2_temp.append(r2_score(y_test, ypred_test))

    em = pd.DataFrame({"train_r2": train_r2_temp, "cv": cv_temp, "test_r2": test_r2_temp})
    gm = em[(abs(em["test_r2"] - em["cv"]) <= 0.05)]
    rs = gm[gm["test_r2"] == gm["test_r2"].max()].index.tolist()[0]
    best_random_state[name] = rs

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=rs)
    x_train_scaled, x_test_scaled = scaler.fit_transform(x_train), scaler.transform(x_test)

    if isinstance(model, (KNeighborsRegressor, SVR, DecisionTreeRegressor, RandomForestRegressor,
                          GradientBoostingRegressor, AdaBoostRegressor, XGBRegressor)):
        param_grid = {}
        if isinstance(model, KNeighborsRegressor):
            param_grid = {"n_neighbors": list(range(1, 50))}
        elif isinstance(model, SVR):
            param_grid = {"C": [0.1, 0.01, 1, 10, 100], "kernel": ["linear", "rbf", "sigmoid", "poly"]}
        elif isinstance(model, DecisionTreeRegressor):
            param_grid = {"criterion": ["squared_error", "friedman_mse"], "max_depth": list(range(1, 20))}
        elif isinstance(model, RandomForestRegressor):
            param_grid = {"n_estimators": list(range(1, 20))}
        elif isinstance(model, GradientBoostingRegressor):
            param_grid = {"n_estimators": list(range(50, 201, 50)), "learning_rate": np.arange(0.1, 1.1, 0.1)}
        elif isinstance(model, AdaBoostRegressor):
            param_grid = {"n_estimators": list(range(1, 21))}
        elif isinstance(model, XGBRegressor):
            param_grid = {"n_estimators": list(range(50, 201, 50)), "learning_rate": np.arange(0.1, 1.1, 0.1)}

        random_search = RandomizedSearchCV(model, param_distributions=param_grid, scoring="r2", cv=5, n_iter=10, random_state=rs)
        random_search.fit(x_train_scaled, y_train)
        best_params[name] = random_search.best_params_
        model = random_search.best_estimator_

    model.fit(x_train_scaled, y_train)
    ypred_train, ypred_test = model.predict(x_train_scaled), model.predict(x_test_scaled)
    train_r2[name], test_r2[name] = r2_score(y_train, ypred_train), r2_score(y_test, ypred_test)
    cv_score_train[name] = cross_val_score(model, x_train_scaled, y_train, cv=5, scoring='r2').mean()
    train_rmse[name], test_rmse[name] = np.sqrt(mean_squared_error(y_train, ypred_train)), np.sqrt(mean_squared_error(y_test, ypred_test))
    train_mae[name], test_mae[name] = mean_absolute_error(y_train, ypred_train), mean_absolute_error(y_test, ypred_test)
    train_mse[name], test_mse[name] = mean_squared_error(y_train, ypred_train), mean_squared_error(y_test, ypred_test)

    pca = PCA(n_components=0.95)
    x_train_pca, x_test_pca = pca.fit_transform(x_train_scaled), pca.transform(x_test_scaled)
    model.fit(x_train_pca, y_train)
    ypred_train_pca, ypred_test_pca = model.predict(x_train_pca), model.predict(x_test_pca)
    train_r2_pca[name], test_r2_pca[name] = r2_score(y_train, ypred_train_pca), r2_score(y_test, ypred_test_pca)
    cv_score_train_pca[name] = cross_val_score(model, x_train_pca, y_train, cv=5, scoring='r2').mean()
    train_rmse_pca[name], test_rmse_pca[name] = np.sqrt(mean_squared_error(y_train, ypred_train_pca)), np.sqrt(mean_squared_error(y_test, ypred_test_pca))
    train_mae_pca[name], test_mae_pca[name] = mean_absolute_error(y_train, ypred_train_pca), mean_absolute_error(y_test, ypred_test_pca)
    train_mse_pca[name], test_mse_pca[name] = mean_squared_error(y_train, ypred_train_pca), mean_squared_error(y_test, ypred_test_pca)

# Output
print("*" * 50 + "\nREGRESSION MODELS\n" + "*" * 50 + "\n")
for name in models.keys():
    print(f"Model: {name}")
    print(f"Best Params: {best_params.get(name, 'N/A')}")
    print(f"Best Random State: {best_random_state[name]}")
    print(f"Without PCA:\nTrain R2: {train_r2[name]}, Test R2: {test_r2[name]}")
    print(f"Train RMSE: {train_rmse[name]}, Test RMSE: {test_rmse[name]}")
    print(f"Train MAE: {train_mae[name]}, Test MAE: {test_mae[name]}")
    print(f"With PCA:\nTrain R2: {train_r2_pca[name]}, Test R2: {test_r2_pca[name]}")
    print(f"Train RMSE: {train_rmse_pca[name]}, Test RMSE: {test_rmse_pca[name]}")
    print("_" * 50)


**************************************************
REGRESSION MODELS
**************************************************

Model: Linear Regression
Best Params: N/A
Best Random State: 44
Without PCA:
Train R2: 0.7415878905353078, Test R2: 0.7817412403741696
Train RMSE: 6184.01335681926, Test RMSE: 5535.186173723055
Train MAE: 4263.391548370806, Test MAE: 3909.169475971726
With PCA:
Train R2: 0.7415878905353078, Test R2: 0.7817412403741695
Train RMSE: 6184.01335681926, Test RMSE: 5535.186173723056
__________________________________________________
Model: Ridge Regression
Best Params: N/A
Best Random State: 44
Without PCA:
Train R2: 0.7415331817358406, Test R2: 0.7818629835472684
Train RMSE: 6184.667935357947, Test RMSE: 5533.642214786479
Train MAE: 4259.9929902622125, Test MAE: 3898.085172878716
With PCA:
Train R2: 0.7415331817358406, Test R2: 0.7818629835472684
Train RMSE: 6184.6679353579475, Test RMSE: 5533.642214786477
__________________________________________________
Model: Lasso Reg

# Model: Gradient Boosting
- Best Params: {'n_estimators': 100, 'learning_rate': 0.1}
- Best Random State: 86
- Without PCA:
- Train R2: 0.8954092145102331, Test R2: 0.8906496617160731
- Train RMSE: 3957.5844388779624, Test RMSE: 3807.4377333112434
- Train MAE: 2229.1873694473957, Test MAE: 2205.62669385190

# The model explains around 89% of the variance on both training and test sets, indicating a great fit and good generalization to new data.
# The Root Mean Square Error (RMSE) on both train and test sets is fairly close, which means the model’s predictions are consistently accurate across both sets.
# The Mean Absolute Error (MAE) is also similar for both sets, showing that on average, your predictions are off by about 2200 currency units.